In [39]:
# Part 1: Initialization and Loading PGN Files

# Import required libraries
import chess
import chess.engine
import chess.pgn
import subprocess
import os
import json
import csv
from IPython.display import display

# Paths to your engines and Maia models
stockfish_path = "/opt/homebrew/bin/stockfish"
maia_model_paths = {
    1100: "maia_weights/maia-1100.pb.gz",
    1200: "maia_weights/maia-1200.pb.gz",
    1300: "maia_weights/maia-1300.pb.gz",
    1400: "maia_weights/maia-1400.pb.gz",
    1500: "maia_weights/maia-1500.pb.gz",
    1600: "maia_weights/maia-1600.pb.gz",
    1700: "maia_weights/maia-1700.pb.gz",
    1800: "maia_weights/maia-1800.pb.gz",
    1900: "maia_weights/maia-1900.pb.gz",
}

# Initialize Stockfish engine
print("Initializing Stockfish engine...")
stockfish_engine = chess.engine.SimpleEngine.popen_uci(stockfish_path)
print("Stockfish engine initialized.")

# Initialize Maia engines
print("Initializing Maia engines...")
maia_engines = {}
for elo, path in maia_model_paths.items():
    maia_engines[elo] = chess.engine.SimpleEngine.popen_uci(["lc0", f"--weights={path}"])
    print(f"Maia {elo} engine initialized.")
print("Maia engines initialized.")
# Function to load games from a PGN file
def load_games_from_pgn(file_path):
    games = []
    with open(file_path, 'r') as pgn_file:
        while True:
            game = chess.pgn.read_game(pgn_file)
            if game is None:
                break
            games.append(game)
    print(f"Loaded {len(games)} games from PGN file {file_path}.")
    return games

Initializing Stockfish engine...


<UciProtocol (pid=18149)>: stderr >>        _
<UciProtocol (pid=18149)>: stderr >> |   _ | |
<UciProtocol (pid=18149)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=18151)>: stderr >>        _
<UciProtocol (pid=18151)>: stderr >> |   _ | |
<UciProtocol (pid=18151)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=18152)>: stderr >>        _
<UciProtocol (pid=18152)>: stderr >> |   _ | |
<UciProtocol (pid=18152)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=18153)>: stderr >>        _
<UciProtocol (pid=18153)>: stderr >> |   _ | |
<UciProtocol (pid=18153)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=18154)>: stderr >>        _
<UciProtocol (pid=18154)>: stderr >> |   _ | |
<UciProtocol (pid=18154)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=18155)>: stderr >>        _
<UciProtocol (pid=18155)>: stderr >> |   _ | |
<UciProtocol (pid=18155)>: 

Stockfish engine initialized.
Initializing Maia engines...
Maia 1100 engine initialized.
Maia 1200 engine initialized.
Maia 1300 engine initialized.
Maia 1400 engine initialized.
Maia 1500 engine initialized.
Maia 1600 engine initialized.
Maia 1700 engine initialized.
Maia 1800 engine initialized.
Maia 1900 engine initialized.
Maia engines initialized.


In [40]:
# Function to find classic puzzles in a game with enhanced debugging
def find_classic_puzzles_with_debug(game, stockfish_engine, threshold=200):
    board = game.board()
    puzzles = []

    for move in game.mainline_moves():
        fen_before = board.fen()
        player = board.turn  # True for White, False for Black

        print(f"Analyzing position: {fen_before}")

        # Analyze the position before the move
        info = stockfish_engine.analyse(board, chess.engine.Limit(time=0.1), multipv=2)
        if len(info) < 2:
            print(f"Insufficient moves found in analysis.")
            continue

        best_move = info[0]["pv"][0]
        best_eval = info[0]["score"].relative.score(mate_score=10000)
        second_best_eval = info[1]["score"].relative.score(mate_score=10000)

        print(f"Best move: {best_move}, Eval: {best_eval}, Second best move eval: {second_best_eval}")

        # Check if the difference is large enough
        if abs(best_eval - second_best_eval) >= threshold:
            print(f"Eval difference is large enough: {abs(best_eval - second_best_eval)}")

            board.push(best_move)
            is_capture = board.is_capture(best_move)
            board.pop()

            print(f"Move {best_move} is a capture: {is_capture}")

            if not is_capture:
                puzzles.append((fen_before, best_move, best_eval, second_best_eval))
                print(f"Puzzle found: {fen_before}, Best move: {best_move}, Eval difference: {abs(best_eval - second_best_eval)}")
            else:
                print(f"Best move is a simple capture, skipping: {best_move}")
        else:
            print(f"Eval difference is too small: {abs(best_eval - second_best_eval)}")

        board.push(move)

    print(f"Total puzzles found in game: {len(puzzles)}")
    return puzzles

# Function to load games from a PGN file
def load_games_from_pgn(file_path):
    games = []
    with open(file_path, 'r') as pgn_file:
        while True:
            game = chess.pgn.read_game(pgn_file)
            if game is None:
                break
            games.append(game)
    print(f"Loaded {len(games)} games from PGN file {file_path}.")
    return games

# Function to evaluate Maia easiness
def evaluate_maia_easiness(fen, best_move, maia_engines):
    board = chess.Board(fen)
    easiness_scores = {}

    for elo, engine in maia_engines.items():
        result = engine.analyse(board, chess.engine.Limit(nodes=1))
        if "pv" in result and best_move in result["pv"]:
            easiness_scores[elo] = result["pv"].index(best_move) / len(result["pv"])

    return easiness_scores

# Function to sort and save results
def sort_and_save_results(puzzles, output_dir="results/classic_puzzles"):
    os.makedirs(output_dir, exist_ok=True)
    puzzles.sort(key=lambda x: max(x["maia_easiness"].values(), default=1), reverse=True)

    csv_file_path = os.path.join(output_dir, "classic_puzzles.csv")
    json_file_path = os.path.join(output_dir, "classic_puzzles.json")

    with open(csv_file_path, "w", newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(["FEN", "Best Move", "Best Eval", "Second Best Eval", "Maia Easiness"])
        for puzzle in puzzles:
            maia_easiness = max(puzzle["maia_easiness"].values(), default=1)
            csvwriter.writerow([puzzle["fen"], puzzle["best_move"], puzzle["best_eval"], puzzle["second_best_eval"], maia_easiness])

    with open(json_file_path, "w") as jsonfile:
        json.dump(puzzles, jsonfile, indent=4)

# Load PGN files and find classic puzzles
pgn_directory = "analysis_pgns_test"
all_puzzles = []

print("Starting to find classic puzzles in PGN files...")

for pgn_file in os.listdir(pgn_directory):
    if pgn_file.endswith(".pgn"):
        pgn_file_path = os.path.join(pgn_directory, pgn_file)
        print(f"Analyzing {pgn_file_path}...")
        all_games = load_games_from_pgn(pgn_file_path)

        for game in all_games:
            print(f"Analyzing game {game.headers.get('Event', 'Unknown Event')}")
            puzzles = find_classic_puzzles_with_debug(game, stockfish_engine)
            for fen, best_move, best_eval, second_best_eval in puzzles:
                maia_easiness = evaluate_maia_easiness(fen, best_move, maia_engines)
                all_puzzles.append({
                    "fen": fen,
                    "best_move": best_move,
                    "best_eval": best_eval,
                    "second_best_eval": second_best_eval,
                    "maia_easiness": maia_easiness
                })

            print(f"Total puzzles after game: {len(all_puzzles)}")

# Sort and save results
sort_and_save_results(all_puzzles)
print("Classic puzzles analysis complete. Results saved.")


Starting to find classic puzzles in PGN files...
Analyzing analysis_pgns_test/4c6eS6SH---1969 Petrosian vs. Spassky .pgn...
Loaded 4 games from PGN file analysis_pgns_test/4c6eS6SH---1969 Petrosian vs. Spassky .pgn.
Analyzing game FIDE World Championship Match 1969
Analyzing position: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
Best move: e2e4, Eval: 33, Second best move eval: 23
Eval difference is too small: 10
Analyzing position: rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1
Best move: c7c5, Eval: -32, Second best move eval: -37
Eval difference is too small: 5
Analyzing position: rnbqkbnr/pp1ppppp/8/2p5/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2
Best move: g1f3, Eval: 26, Second best move eval: 24
Eval difference is too small: 2
Analyzing position: rnbqkbnr/pp1ppppp/8/2p5/4P3/5N2/PPPP1PPP/RNBQKB1R b KQkq - 1 2
Best move: d7d6, Eval: -32, Second best move eval: -40
Eval difference is too small: 8
Analyzing position: rnbqkbnr/pp1p1ppp/4p3/2p5/4P3/5N2/PPPP1PPP/RNBQKB